In [1]:
import keras
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import normalize
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Using TensorFlow backend.


## Features

In [2]:
features = pd.read_csv("./data/NUSW-NB15_features.csv", 
                       engine='python', 
                       encoding = "ISO-8859-1").values
features = features[:-2,:]
feature_names = features[:, 1]
feature_types = np.array([item.lower() for item in features[: ,2]])

### Filters

In [3]:
nominal_cols = np.where(feature_types == "nominal")[0]
integer_cols = np.where(feature_types == "integer")[0]
binary_cols = np.where(feature_types == "binary")[0]
float_cols = np.where(feature_types == "float")[0]

In [4]:
nominal_names = feature_names[nominal_cols]
integer_names = feature_names[integer_cols]
binary_names = feature_names[binary_cols]
float_names = feature_names[float_cols]

In [5]:
print(nominal_names)
print(integer_names)
print(binary_names)
print(float_names)

['srcip' 'dstip' 'proto' 'state' 'service']
['sport' 'dsport' 'sbytes' 'dbytes' 'sttl' 'dttl' 'sloss' 'dloss' 'Spkts'
 'Dpkts' 'swin' 'dwin' 'stcpb' 'dtcpb' 'smeansz' 'dmeansz' 'trans_depth'
 'res_bdy_len' 'ct_state_ttl' 'ct_flw_http_mthd' 'ct_ftp_cmd' 'ct_srv_src'
 'ct_srv_dst' 'ct_dst_ltm' 'ct_src_ ltm' 'ct_src_dport_ltm'
 'ct_dst_sport_ltm' 'ct_dst_src_ltm']
['is_sm_ips_ports' 'is_ftp_login']
['dur' 'Sload' 'Dload' 'Sjit' 'Djit' 'Sintpkt' 'Dintpkt' 'tcprtt' 'synack'
 'ackdat']


## Read Data

In [6]:
data1 = pd.read_csv("./data/UNSW-NB15_1.csv", header=None)
data2 = pd.read_csv("./data/UNSW-NB15_2.csv", header=None)
data3 = pd.read_csv("./data/UNSW-NB15_3.csv", header=None)
data4 = pd.read_csv("./data/UNSW-NB15_4.csv", header=None)

/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,39,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data = pd.concat([data1, data2, data3, data4])
del data1
del data2
del data3
del data4

In [8]:
data[integer_cols] = data[integer_cols].apply(pd.to_numeric, args=('coerce',))
data[binary_cols] = data[binary_cols].apply(pd.to_numeric, args=('coerce',))
data[float_cols] = data[float_cols].apply(pd.to_numeric, args=('coerce',))
data[48] = data[48].apply(pd.to_numeric, args=('coerce',))

In [9]:
data.loc[:,47] = data.loc[:,47].replace(np.nan, "normal", regex=True)\
                            .apply(lambda x: x.strip().lower())

data.loc[:, binary_cols] = data.loc[:, binary_cols] \
                               .replace(np.nan, 0, regex=True)

data.loc[:,37:39] = data.loc[:,37:39].replace(np.nan, 0, regex=True)

data.loc[:,nominal_cols] = data.loc[:,nominal_cols] \
                               .applymap(lambda x: x.strip().lower())

In [10]:
data.loc[:,47] = data.loc[:,47] \
                        .replace('backdoors','backdoor', regex=True) \
                        .apply(lambda x: x.strip().lower())

In [11]:
data_np = data.values
del data

In [12]:
nominal_x = data_np[:, nominal_cols][:,:]
integer_x = data_np[:, integer_cols][:,:].astype(np.float32)
binary_x = data_np[:, binary_cols][:,:].astype(np.float32)
float_x = data_np[:, float_cols][:,:].astype(np.float32)

In [13]:
v = DictVectorizer(sparse=False)
D = map(lambda dataline: dict(zip(nominal_names, dataline)), nominal_x)
labeled_nominal_x = v.fit_transform(D).astype(np.float32)
del nominal_x

In [14]:
X = np.concatenate((integer_x, labeled_nominal_x, float_x, binary_x), axis=1)

del integer_x
del labeled_nominal_x
del float_x
del binary_x

In [15]:
nan_indices = []
for feature_i in range(X.shape[1]):
    nan_indices.extend(list(np.where(np.isnan(X[:, feature_i]))[0]))
nan_indices = np.unique(nan_indices)

In [16]:
nan_indices.shape

(308,)

In [17]:
X_no_nans = np.delete(X, nan_indices, axis=0)
del X

In [18]:
normalized_X = normalize(X_no_nans, copy=False)

del X_no_nans

In [19]:
Y = np.delete(data_np[:, -2], nan_indices)
Y_A = np.delete(data_np[:, -1], nan_indices).astype(np.int16)
del data_np

In [20]:
D = [{'attack_cat': y} for y in Y]
labeled_Y = v.fit_transform(D)
del D

# Backup

In [21]:
import h5py
h5f = h5py.File('data.h5', 'w')
h5f.create_dataset('normalized_X', data=normalized_X)
h5f.create_dataset('labeled_Y', data=labeled_Y)
dt = h5py.special_dtype(vlen=str)
h5f.create_dataset('Y', data=Y, dtype=dt)
h5f.create_dataset('Y_A', data=Y_A)
h5f.close()

del Y
del normalized_X
del labeled_Y

# Reload

In [22]:
import h5py
h5f = h5py.File('data.h5', 'r')
normalized_X = h5f['normalized_X'].value
labeled_Y = h5f['labeled_Y'].value
Y = h5f['Y'].value
Y_A = h5f['Y_A'].value
h5f.close()

/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  after removing the cwd from sys.path.
/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """
/home/ozhan/Desktop/ceng489/tp/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


In [23]:
attack_categories = np.unique(Y)
rf_normal_inds = []
nn_normal_inds = []
inds = []

cat_sizes = []
np.random.seed(1337)
max_training_samples = 5000

for cat in attack_categories:
    indices = np.ix_(Y == cat)[0]
    np.random.shuffle(indices)
    if cat == 'normal':
        len_of_subset = min(np.floor(len(indices)*0.9), 100000)
        rf_normal_inds = indices[:len_of_subset]
        cat_size = len(indices)
        print(cat, ': training samples =', len_of_subset, '| total samples =', cat_size)
    else:
        len_of_subset = min(np.floor(len(indices)*0.9), max_training_samples)
        cat_size = len(indices)
        print(cat, ': training samples =', len_of_subset, '| total samples =', cat_size)
        cat_sizes.append(cat_size)
        inds.extend(indices[:int(len_of_subset)])

analysis : training samples = 2409.0 | total samples = 2677
backdoor : training samples = 2096.0 | total samples = 2329
dos : training samples = 5000 | total samples = 16353
exploits : training samples = 5000 | total samples = 44525
fuzzers : training samples = 5000 | total samples = 24246
generic : training samples = 5000 | total samples = 215481
normal : training samples = 100000 | total samples = 2218456
reconnaissance : training samples = 5000 | total samples = 13987
shellcode : training samples = 1359.0 | total samples = 1511
worms : training samples = 156.0 | total samples = 174


In [24]:
rf_inds = []
rf_inds.extend(inds)
rf_inds.extend(rf_normal_inds)
X_rf_train = normalized_X[rf_inds, :]
X_rf_test = np.delete(normalized_X, rf_inds, axis=0)

del normalized_X

Y_rf_train = Y_A[rf_inds]
Y_rf_test = np.delete(Y_A, rf_inds, axis=0)

In [25]:
def find_importances(X_train, Y_train):
    model = ExtraTreesClassifier()
    model = model.fit(X_train, Y_train)
    
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]  # Top ranking features' indices
    return importances, indices, std

def KBest(X, y, k):
    model = SelectKBest(score_func=chi2, k=k)
    model.fit(X,y)
    supp = model.get_support()
    return np.argwhere(supp).flatten()

In [26]:
rf_importances, rf_indices, rf_std = find_importances(X_rf_train, 
                                                      Y_rf_train)

In [27]:
import matplotlib.pyplot as plt
# Plot the feature importances of the forest
def plot_feature_importances(X_train, importances, indices, std, title):
    plt.figure(num=None, figsize=(180, 120), dpi=180, facecolor='w', edgecolor='k')
    plt.title(title)
    width=5
    plt.bar(range(X_train.shape[1]), importances[indices],
          width=5, color="r", yerr=std[indices], align="center") #tagy 1.5 > .8
    plt.xticks(range(X_train.shape[1]), indices)
    plt.xlim([-1, X_train.shape[1]]) # -1 tagy
    plt.show()

In [28]:
plot_feature_importances(X_rf_train, rf_importances, 
                         rf_indices, rf_std, 
                         title='Feature importances')

# Select with kbest

In [29]:
kbest_indices = KBest(X_rf_train, Y_rf_train, 30)

reduced_X_rf_train = X_rf_train[:, kbest_indices]
reduced_Y_rf_train = Y_rf_train

reduced_X_rf_test = X_rf_test[:, kbest_indices]
reduced_Y_rf_test = Y_rf_test

# Select with feature importance

In [30]:
NB_RF_FEATURES = 30

reduced_X_rf_train = X_rf_train[:, rf_indices[0:NB_RF_FEATURES]]
reduced_Y_rf_train = Y_rf_train

reduced_X_rf_test = X_rf_test[:, rf_indices[0:NB_RF_FEATURES]]
reduced_Y_rf_test = Y_rf_test

# Adaboost

In [31]:
clf = AdaBoostClassifier(n_estimators=100, 
                         random_state=0, 
                         learning_rate=1)
clf = clf.fit(reduced_X_rf_train, reduced_Y_rf_train)
pred_Y = clf.predict(reduced_X_rf_test)

print(classification_report(reduced_Y_rf_test, pred_Y))
print(confusion_matrix(reduced_Y_rf_test, pred_Y))
score = accuracy_score(reduced_Y_rf_test, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       0.97      0.93      0.95   2118456
           1       0.61      0.82      0.70    290263

    accuracy                           0.91   2408719
   macro avg       0.79      0.88      0.82   2408719
weighted avg       0.93      0.91      0.92   2408719

[[1964361  154095]
 [  51247  239016]]
0.9147505375263781


# ExtraTreesClassifier

In [32]:
clf = ExtraTreesClassifier(n_estimators=31, criterion='entropy')
clf = clf.fit(reduced_X_rf_train, reduced_Y_rf_train)

pred_Y = clf.predict(reduced_X_rf_test)

print(classification_report(reduced_Y_rf_test, pred_Y))
print(confusion_matrix(reduced_Y_rf_test, pred_Y))
score = accuracy_score(reduced_Y_rf_test, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99   2118456
           1       0.91      1.00      0.95    290263

    accuracy                           0.99   2408719
   macro avg       0.95      0.99      0.97   2408719
weighted avg       0.99      0.99      0.99   2408719

[[2089201   29255]
 [    769  289494]]
0.9875352832771278


# KNN

In [33]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier()
neigh.fit(reduced_X_rf_train, reduced_Y_rf_train)
knn_X_set = reduced_X_rf_test[:390000]
knn_Y_set = reduced_Y_rf_test[:390000]
pred_Y = neigh.predict(knn_X_set)

print(classification_report(knn_Y_set, pred_Y))
print(confusion_matrix(knn_Y_set, pred_Y))
score = accuracy_score(knn_Y_set, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       0.99      0.95      0.97    371778
           1       0.44      0.79      0.56     18222

    accuracy                           0.94    390000
   macro avg       0.71      0.87      0.77    390000
weighted avg       0.96      0.94      0.95    390000

[[353479  18299]
 [  3890  14332]]
0.9431051282051282


# DecisionTreeClassifier

In [34]:
clf = DecisionTreeClassifier()
clf = clf.fit(reduced_X_rf_train, reduced_Y_rf_train)
pred_Y = clf.predict(reduced_X_rf_test)

print(classification_report(reduced_Y_rf_test, pred_Y))
print(confusion_matrix(reduced_Y_rf_test, pred_Y))
score = accuracy_score(reduced_Y_rf_test, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99   2118456
           1       0.89      0.97      0.93    290263

    accuracy                           0.98   2408719
   macro avg       0.94      0.98      0.96   2408719
weighted avg       0.98      0.98      0.98   2408719

[[2082360   36096]
 [   8132  282131]]
0.9816383729276847


# RandomForestClassifier

In [35]:
clf = RandomForestClassifier()
clf = clf.fit(reduced_X_rf_train, reduced_Y_rf_train)
pred_Y = clf.predict(reduced_X_rf_test)

print(classification_report(reduced_Y_rf_test, pred_Y))
print(confusion_matrix(reduced_Y_rf_test, pred_Y))
score = accuracy_score(reduced_Y_rf_test, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99   2118456
           1       0.88      0.97      0.92    290263

    accuracy                           0.98   2408719
   macro avg       0.94      0.97      0.95   2408719
weighted avg       0.98      0.98      0.98   2408719

[[2079956   38500]
 [  10026  280237]]
0.9798540219926027


# LogisticRegression

In [36]:
clf = LogisticRegression(max_iter=1000, random_state=0)
clf = clf.fit(reduced_X_rf_train, reduced_Y_rf_train)
pred_Y = clf.predict(reduced_X_rf_test)

print(classification_report(reduced_Y_rf_test, pred_Y))
print(confusion_matrix(reduced_Y_rf_test, pred_Y))
score = accuracy_score(reduced_Y_rf_test, pred_Y)
print(score)

              precision    recall  f1-score   support

           0       0.98      0.90      0.93   2118456
           1       0.53      0.84      0.65    290263

    accuracy                           0.89   2408719
   macro avg       0.75      0.87      0.79   2408719
weighted avg       0.92      0.89      0.90   2408719

[[1900500  217956]
 [  47612  242651]]
0.8897472058799719
